Nuclear Physics Approach Anomalous Phenomena in E<18 Ke V Hydrogen Ion Beam Implantation Experiments on Pd and Ti WANG Tieshan* PIAO Yubo** HAO Jifang* WANG Xuezhi** JIN Genming* and NIU Zhanqi** *Institute of Modern Physics, Chinese Academy of Sciences Lanzhou 730000, P.R. China **Department of Modern Physics of Lanzhou University Lanzhou 730000, P.R. China


In [143]:
# Imports
import os
import re
import copy
import numpy as np
import pdfplumber
import nltk
import openai
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
pdf_dir = "./data"
pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith(".pdf")]
title_abstract_list = []
print(pdf_files)

# testDir="./data/doc16.pdf"

# print(extract_title_abstract(testDir)[0])

In [ ]:
def extract_title_abstract(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[0]

        text = page.extract_text(x_tolerance=1, y_tolerance=5)

        # print(page.chars)
        table={}
        for x,i in enumerate(page.chars):
            # print(i["text"])
            # if i["text"]==" " or i["text"]=="\n":
            #     print("new line")
            if i["size"] not in table:
                table[i["size"]]=""
            if x>0 and i["x0"]-page.chars[x-1]["x1"]>=1:
                table[i["size"]]+=" "
            if x>0 and page.chars[x-1]["y1"]-i["y1"]>5:
                table[i["size"]]+=" "
            table[i["size"]]+=i["text"]


        # print(text,"\n") 
        tmp=copy.deepcopy(table)

        for i in tmp:
            if len(tmp[i])<30:
                del table[i]

        title=table[max(table.keys())].strip().replace("  "," ")
        abstract_match = re.search(r"Abstract\s+(.*?[.!?])\s*\n[c,©]", text, re.DOTALL)
        if not abstract_match:
            abstract_match = re.search(r'(?i)abstract\b\.?:?]?\s+((?:.|\n)+?)(?=\n\d+\.|Key|Introduction|[0,1]?\Z)', text, re.DOTALL)

        if abstract_match:
            abstract = abstract_match.group(1).strip()

    if not abstract_match:
        abstract = "No abstract found"
    # abstract = text.split('\n')
    # print("Title: ",title,"Abstract: ",abstract)
    # print(title, "\n")

    return title, abstract

# 1B

In [ ]:


def get_text_from_pdf(pdf_path):
    allText=""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
             allText+=page.extract_text(x_tolerance=1, y_tolerance=5)
    return allText

lemmatizer=WordNetLemmatizer()


def preprocess_text(text):
    words=word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered= [lemmatizer.lemmatize(w.lower()) for w in words if not w in stop_words and w.isalpha()]
    return filtered


In [220]:
embeddings={}

def get_embedding(text, model="text-embedding-ada-002"):
   return openai.Embedding.create(input = text, model=model)['data'][0]['embedding']

# embs=get_embedding("\n".join(filtered))

for i in pdf_files:
    text=get_text_from_pdf("./data/"+i)
    filtered=" ".join(preprocess_text(text))
    if len(filtered)>8192:
        filtered=filtered[:8192]
    embeddings[i]=get_embedding(filtered)

# aaa=get_text_from_pdf("./data/doc26.pdf")
# aaa2=preprocess_text(aaa)
# print(aaa2)
# aaa3=get_embedding(aaa2)
# print(aaa3)
# # get the mean of aaa3
# print(np.mean(aaa3,axis=0))
    

In [230]:
avg_embedding=[np.mean(np.array(embeddings[i]), axis=0) for i in embeddings]
avg_embedding=np.array(avg_embedding)


In [234]:
print(cosine_similarity(embeddings["doc11.pdf"],embeddings["doc16.pdf"]))

0.8469229574089849


In [249]:
import random

random_indices = random.sample(range(len(pdf_files)), 5)
random_indices

[0, 11, 12, 18, 24]

In [250]:
random_embeddings = np.array([embeddings[pdf_files[i]] for i in random_indices])
random_documents = [pdf_files[i] for i in random_indices]


In [251]:
for i,j in zip(random_documents,random_embeddings):
    # Exclude self-similarity by setting it to a negative value
    print('Most similar documents to Document', i, 'are:')
    table={}
    for k,l in embeddings.items():
        if k!=i:
            table[k]=cosine_similarity(j,l)
    table=sorted(table.items(), key=lambda x: x[1], reverse=True)
    for k in table[:3]:
        print("Document:",k[0],"Similarity:",k[1])

    # similarities = cosine_similarity(j.reshape(1,-1), avg_embedding)[0]
    # most_similar_indices = np.argsort(similarities)[-3:]
    # for j, idx in enumerate(reversed(most_similar_indices), 1):
    #     print(f'{j}. Document {idx + 1} (Similarity: {similarities[idx]:.4f})')


Most similar documents to Document doc1.pdf are:
Document: doc8.pdf Similarity: 0.923830645284825
Document: doc2.pdf Similarity: 0.917085091935906
Document: doc7.pdf Similarity: 0.9124823522910969
Most similar documents to Document doc2.pdf are:
Document: doc1.pdf Similarity: 0.917085091935906
Document: doc28.pdf Similarity: 0.9075754244280272
Document: doc30.pdf Similarity: 0.8940143192454763
Most similar documents to Document doc20.pdf are:
Document: doc23.pdf Similarity: 0.9193531869487045
Document: doc7.pdf Similarity: 0.918974585433316
Document: doc19.pdf Similarity: 0.9169125824110804
Most similar documents to Document doc26.pdf are:
Document: doc28.pdf Similarity: 0.9361660868718512
Document: doc27.pdf Similarity: 0.9075055382574022
Document: doc29.pdf Similarity: 0.9065079389940269
Most similar documents to Document doc31.pdf are:
Document: doc29.pdf Similarity: 0.8995850871082629
Document: doc27.pdf Similarity: 0.892087128155389
Document: doc28.pdf Similarity: 0.88635422972221

# Word2Vec

w2VModel = Word2Vec(filtered, min_count = 1, vector_size = 100, window = 5, workers = 4)


all_vectors = [w2V.wv[word] for word in model.wv.key_to_index]
mean_vector = np.mean(all_vectors, axis=0)

print("Mean vector:", mean_vector)